In [1]:
!pip install langchain
!pip install boto3
!pip install langchain-community
!pip install langchain-aws
!pip install gpt4all
!pip install faiss-gpu

In [2]:
# !pip install aws configure
# !pip install awscli

# !aws configure

In [3]:
import boto3
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.gpt4all import GPT4AllEmbeddings
from langchain_aws.llms.bedrock import Bedrock
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA

In [4]:
from google.colab import userdata
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
def get_bedrock_runtime(region_name, *args, **kwargs):
    return boto3.client(service_name='bedrock-runtime', region_name=region_name, *args, **kwargs)


def get_langchain_bedrock_llm(model_id, client, *args, **kwargs):
    return Bedrock(model_id=model_id, client=client, *args, **kwargs)

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=900,
    chunk_overlap=20,
    length_function=len
)

In [7]:
vectorstoredir = '/content/drive/MyDrive/Apologetics Vectorstore'
embed_model_name = 'all-MiniLM-L6-v2.gguf2.f16.gguf'
llm_name = 'mistral.mixtral-8x7b-instruct-v0:1'

In [10]:
aws_access_token = userdata.get('AWS_STEPHENHO_SERVERLESS_ACC_ACCESS_TOKEN')
aws_secret_key = userdata.get('AWS_STEPHENHO_SERVERLESS_ACC_SECRET_ACCESS_KEY')

bedrock_runtime = get_bedrock_runtime('us-east-1', aws_access_key_id=aws_access_token, aws_secret_access_key=aws_secret_key)
llm = get_langchain_bedrock_llm(llm_name, bedrock_runtime)

In [11]:
embedding_model = GPT4AllEmbeddings(model_name=embed_model_name)

In [12]:
db = FAISS.load_local(vectorstoredir, embedding_model, allow_dangerous_deserialization=True)
retriever = db.as_retriever()

In [13]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever, return_source_documents=True)

In [14]:
qa({'query': 'What is the ontological argument for the existence of God?'})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'query': 'What is the ontological argument for the existence of God?',
 'result': ' The ontological argument is a philosophical argument for the existence of God that argues from the concept of God to the existence of God. It comes in two forms: the Perfect Being form and the Necessary Being form. The Perfect Being form argues that existence is a perfection and since God is by definition a perfect being, God must exist. The Necessary Being form argues that it is logically necessary to affirm what is necessary of the concept of a Necessary Being, and since existence is logically necessary to the concept of a Necessary Being, a Necessary Being (God) necessarily exists. The ontological argument has had a long and illustrious career and is still a topic of debate among philosophers and theologians.',
 'source_documents': [Document(page_content='ontological argument (though not the first to call it this) was Anselm (1033–1109). In its simplest form it\nargues from the idea of God to the ex